In [5]:
! pip install transformers

from transformers import AutoTokenizer, AutoModelForMaskedLM
from google.colab import drive
from pandas import DataFrame
import pandas as pd
import os

drive.mount('/content/drive')

folder = "drive/Othercomputers/Il mio Laptop/Universita/[IA] Artificial Intelligence/[HLT] Human Language Technologies/project/"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
dataset = pd.read_csv(folder+"Sources/dataset.csv")

In [7]:
dataset

,text,labels
0,Non riferisce familiarità diabetica .,O O O O O
1,Fumatrice .,O O
2,"Non ipertensione , dislipidemia o diabete .",O B-DISO O B-DISO O B-DISO O
3,Nel 1999 protesi d' anca sinistra e destra .,O O O O O O O O O
4,Nel 2003 operata di sostituzione valvolare per...,O O O O O O O B-DISO I-DISO O O O O O O O O O O
...,...,...
169368,Coumadin :,O O
169369,lunedi p.v. eseguire prelievo per INR. Fragmin :,O O O O O O O O
169370,Tra circa 3 mesi ripetere un ecg dinamico di 2...,O O O O O O B-TREA O O O O O O O
169371,Setto nasale lievemente deviato verso sn. Rino...,B-BODY I-BODY O O O O O O


In [12]:
dataset[dataset.text.str.split().str.len() > 512]

,text,labels
128677,"Ritornare poi a Lasix 500 mg cpr , 1 / 2 cpr a...",O O O B-DRUG I-DRUG O O O O O O O O O O B-DRU...
166607,"Ritornare poi a Lasix 500 mg cpr , 1 / 2 cpr a...",O O O O O O O O O O O O O O O O O O O O O O O...


In [13]:
dataset.iloc[166607].values

array(["Ritornare poi a Lasix 500 mg cpr , 1 / 2 cpr al mattino e Lasix 25 mg cpr 3 cpr ore 14 Plaquenil 1 cpr / die Allopurinolo 300 mg cpr , 1 / 2 cpr / die Lanoxin 0.125 mg cpr , 1 cpr al dì Peridon 10 mg cpr , 1 cpr mezz' ora prima dei pasti principali Pantorc 40 mg cpr , 1 cpr la sera Nespo 1 fl s.c. alla settimana Ossigeno-terapia con flusso di O2 di 3 lt / min per 24 ore , aumentando a 5 L / min sotto sforzo Aliflus 25/250 spray , 2 inalazione mattina e sera con distanziatore Aerochamber ( sciacquare bene il cavo orale dopo l' inalazione ) Acetilcisteina 600 mg cpr eff 1 cpr 2 volte al giorno a cicli di 10 giorni al mese Medrol 16 mg cpr , 1 / 4 cpr a giorni alterni Cardioaspirina 100 mg cpr , 1 cpr / die Elopram 20 mg cpr , 1 cpr al mattino Volibris 5 mg 1 cpr / die Revatio 20 mg cpr , 1 cpr x 3 / die Ventavis 5 mcg fl , 1 fl per 6 inazioni al giorno Aldactone 25 mg cpr , 1 cpr / die Lasix 500 mg cpr , 1 / 2 cpr al mattino e Lasix 25 mg cpr 3 cpr ore 14 Plaquenil 1 cpr / die Al

In [ ]:
unique_labels = set()

for lb in [i.split() for i in dataset['labels'].values.tolist()]:
  [unique_labels.add(i) for i in lb if i not in unique_labels]
 
labels_to_ids = {k: v for v, k in enumerate(sorted(unique_labels))}
ids_to_labels = {v: k for v, k in enumerate(sorted(unique_labels))}

print(unique_labels)
print(labels_to_ids)

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-italian-xxl-cased")

def align_label(text : str, labels : str):

  # Trasforming a string of label into array
  labels = labels.split(" ")

  # tokenize the input
  token = tokenizer(text,padding='max_length', max_length = 512, truncation=True, return_tensors="pt")
  # return a array of token id, if some words are splitted the id is repeated 
  word_ids = token.word_ids()
  # We take into cosideration the previous word to identify if the id is already seen
  previous_id = None
  # Aligned labels
  label_ids = []

  # We can all ids in the token and we try to associate to a label  
  for word_idx in word_ids:

    # typically when we encounter [CLS]
    if word_idx is None: 
      label_ids.append(-100)

    # In this case the words are different
    elif word_idx != previous_id:
      try:
        # We try to associate a label
        label_ids.append(labels_to_ids[labels[word_idx]])
      except:
        label_ids.append(-100)

    # The default case is when the id is the same so this means the we have 
    # a piece of the same word, so we adding the same label id
    else:
      # Also in this case we try to associate a label
      try:
        label_ids.append(labels_to_ids[labels[word_idx]])
      except:
        label_ids.append(-100)
    
    previous_id = word_idx 
    
  return label_ids

{'I-DISO', 'B-SIGN', 'I-TREA', 'I-SIGN', 'I-ACTI', 'B-DRUG', 'B-DISO', 'I-DRUG', 'B-TREA', 'I-BODY', 'O', 'B-BODY', 'B-ACTI'}
{'B-ACTI': 0, 'B-BODY': 1, 'B-DISO': 2, 'B-DRUG': 3, 'B-SIGN': 4, 'B-TREA': 5, 'I-ACTI': 6, 'I-BODY': 7, 'I-DISO': 8, 'I-DRUG': 9, 'I-SIGN': 10, 'I-TREA': 11, 'O': 12}


In [ ]:
text = dataset.iloc[4].text
text

'Nel 2003 operata di sostituzione valvolare per stenosi mitralica a Legnano , con protesi biologica di Carpentier Edwards .'

In [ ]:
label = dataset.iloc[4].labels
label

' O O O O O O O B-DISO I-DISO O O O O O O O O O O'

In [ ]:
tokenizer(text,padding='max_length', max_length = 512, truncation=True, return_tensors="pt")

{'input_ids': tensor([[  102,   542,  5397,  1199,   115,   120, 10072, 21484,   113,   156,
         13139,   410, 18341,   485,   111, 24572, 20096, 30879,  1307,   153,
          1202,   210, 15215,   120,  1237,   941, 19494, 30882, 30356,   697,
           103,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,  

In [ ]:
r = align_label(text,label)
r

[-100,
 -100,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 2,
 2,
 8,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 12,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -100,
 -